In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
import random
import os
import time
from tqdm import tqdm
import datetime

In [2]:
player_df = pd.read_csv('/home/valesco/Datasets/PGA_Data/salary_folder/pga_champ_salaries.csv')
player_df.head()

,player_name,name,salary,owned
0,Jordan Spieth (9168524),Jordan Spieth,12000,0.16
1,Rory McIlroy (9168525),Rory McIlroy,11800,0.23
2,Dustin Johnson (9168526),Dustin Johnson,11400,0.18
3,Rickie Fowler (9168527),Rickie Fowler,10700,0.24
4,Hideki Matsuyama (9168528),Hideki Matsuyama,10500,0.18


In [212]:
nonzero_player_df = player_df.loc[player_df['owned'] > 0]

players = list(nonzero_player_df['name'].values)

pid_dict = {}

for p, player in enumerate(players):
    salary = player_df['salary'].loc[player_df['name'] == player].values[0]
    own = player_df['owned'].loc[player_df['name'] == player].values[0]
    pid_dict[p] = {'name': player, 'salary': salary, 'own': own, 'lineup_diff': 0}

pids = list(pid_dict.keys())
pid_dict

{0: {'lineup_diff': 0, 'name': 'Jordan Spieth', 'own': 0.16, 'salary': 12000},
 1: {'lineup_diff': 0,
  'name': 'Rory McIlroy',
  'own': 0.23000000000000001,
  'salary': 11800},
 2: {'lineup_diff': 0,
  'name': 'Dustin Johnson',
  'own': 0.17999999999999999,
  'salary': 11400},
 3: {'lineup_diff': 0,
  'name': 'Rickie Fowler',
  'own': 0.23999999999999999,
  'salary': 10700},
 4: {'lineup_diff': 0,
  'name': 'Hideki Matsuyama',
  'own': 0.17999999999999999,
  'salary': 10500},
 5: {'lineup_diff': 0,
  'name': 'Jason Day',
  'own': 0.080000000000000002,
  'salary': 10400},
 6: {'lineup_diff': 0, 'name': 'Jon Rahm', 'own': 0.13, 'salary': 10100},
 7: {'lineup_diff': 0,
  'name': 'Henrik Stenson',
  'own': 0.10000000000000001,
  'salary': 9800},
 8: {'lineup_diff': 0,
  'name': 'Sergio Garcia',
  'own': 0.14999999999999999,
  'salary': 9400},
 9: {'lineup_diff': 0,
  'name': 'Brooks Koepka',
  'own': 0.14999999999999999,
  'salary': 9200},
 10: {'lineup_diff': 0, 'name': 'Justin Rose', 'o

In [213]:
def random_combination(iterable, r): 
    #Random selection from combinations
    
    pool = tuple(iterable)
    n = len(pool)
    indices = sorted(random.sample(range(n), r))
    return tuple(pool[i] for i in indices)


def random_lineup_generator(iterable, r, lineups_to_create):
    temp_ls = []
    for i in range(lineups_to_create):
        temp_ls.append(random_combination(iterable, r))
        
    return temp_ls


def lineup_difference (p, player):
    curr_df_len = len(lineup_df)

    adjust_df_len = num_lineups - curr_df_len

    target_own = pid_dict[p]['own']
    target_lus = round(target_own * num_lineups) + adjust_df_len

    actual_lus = count_df['count'].loc[count_df['name'] == player].values[0]

    lu_diff = int(target_lus - actual_lus)
    
    return lu_diff

##Create return_ownership

def return_ownership(player):
    for key, value in pid_dict.items():
        if value['name'] == player:
            return value['own']

#num lineups in tournament
num_lineups = 15000

#cumulative ownership for the field
cum_per_owned = 6
start = datetime.datetime.now()

#generate random num lineups for tournament
random_lineups = random_lineup_generator(players, 6, num_lineups)
    



In [214]:
lineup_df = pd.DataFrame(random_lineups)

#Count players in lineups
count_df = lineup_df.apply(pd.Series.value_counts)

#Turn counts into percent
count_df['count'] = count_df.sum(axis = 1)
count_df.reset_index(inplace = True)
count_df.rename(columns = {'index': 'name'}, inplace = True)
count_df['owned'] = count_df['name'].apply(lambda x: return_ownership(x))
count_df['target_lus'] = count_df['owned'] * num_lineups
count_df['lu_diff'] = 0




for iteration in tqdm(range(8)):
    for p, player in tqdm(enumerate(players)):
        #curr_df_len = len(lineup_df)

        #adjust_df_len = num_lineups - curr_df_len

        #target_own = pid_dict[p]['own']
        #target_lus = round(target_own * num_lineups) + adjust_df_len

        #actual_lus = count_df['count'].loc[count_df['name'] == player].values[0]

        #lu_diff = int(target_lus - actual_lus)
        
        lu_diff = lineup_difference(p, player)

        if lu_diff > 0:
            
            #Create list teams of 5 players without current player
            if p < 1:
                temp_player_ls = list(player_df['name'].unique())
                temp_player_ls.remove(player)
                
                if len(temp_player_ls) < 5:
                    print('error')
            else:
                temp_player_ls = list(count_df['name'].loc[count_df['lu_diff'] > 0].unique())
                temp_player_ls.remove(player)
                
            temp_5_man_lus = random_lineup_generator(temp_player_ls, 5, lu_diff)   

            #Add player to 5 man combos
            temp_lus = []
            for t, lu in enumerate(temp_5_man_lus):
                temp_lus.append(lu + (player,))

            temp_df = pd.DataFrame(temp_lus)

            #replace teams in lineup_df

            #select random indexs len num of lineups to replace
            removal_index = sorted(random.sample(range(num_lineups), lu_diff))

            new_df = lineup_df.loc[~lineup_df.index.isin(removal_index)]

            lineup_df = pd.concat([new_df, temp_df])
            lineup_df.reset_index(inplace = True, drop = True)


        if lu_diff < 0:
            lu_diff = abs(lu_diff)

            # Create 6 man teams to replace teams with player
            #temp_player_ls = list(player_df['name'].unique())
            if p < 1:
                temp_player_ls = list(player_df['name'].unique())
                temp_player_ls.remove(player)
                
                if len(temp_player_ls) < 6:
                    print('error')

            else:
                temp_player_ls = list(count_df['name'].loc[count_df['lu_diff'] > 0].unique())
                #temp_player_ls.remove(player)

            temp_6_man_lus = random_lineup_generator(temp_player_ls, 6, lu_diff)
            temp_df = pd.DataFrame(temp_6_man_lus)

            #Remove existing lineups with player in them
            removal_index = lineup_df[lineup_df.apply(lambda x: x.str.contains(player, case = False).any(),
                                                      axis = 1)].sample(frac=1).head(lu_diff).index

            new_df = lineup_df.loc[~lineup_df.index.isin(removal_index)]

            lineup_df = pd.concat([new_df, temp_df])
            lineup_df.reset_index(inplace = True, drop = True)
            
            

        count_df = lineup_df.apply(pd.Series.value_counts)

        #Turn counts into percent
        count_df['count'] = count_df.sum(axis = 1)
        count_df.reset_index(inplace = True)
        count_df.rename(columns = {'index': 'name'}, inplace = True)
        count_df['owned'] = count_df['name'].apply(lambda x: return_ownership(x))
        count_df['target_lus'] = count_df['owned'] * num_lineups
        count_df['lu_diff'] = count_df['target_lus'] - count_df['count']
        
print(datetime.datetime.now() - start)       

  0%|          | 0/8 [00:00<?, ?it/s]
0it [00:00, ?it/s]
3it [00:00, 26.21it/s]
6it [00:00, 27.20it/s]
10it [00:00, 30.02it/s]
14it [00:00, 31.43it/s]
17it [00:02,  3.93it/s]
22it [00:02,  5.40it/s]
26it [00:05,  3.28it/s]
28it [00:07,  1.82it/s]
34it [00:07,  2.56it/s]
37it [00:14,  1.04it/s]
42it [00:16,  1.24it/s]
44it [00:21,  1.24s/it]
46it [00:23,  1.21s/it]
47it [00:25,  1.53s/it]
48it [00:28,  1.75s/it]
54it [00:30,  1.34s/it]
57it [00:32,  1.17s/it]
59it [00:34,  1.16s/it]
61it [00:37,  1.15s/it]
62it [00:39,  1.49s/it]
63it [00:41,  1.72s/it]
64it [00:44,  1.90s/it]
65it [00:46,  2.01s/it]
66it [00:48,  2.08s/it]
68it [00:50,  1.80s/it]
69it [00:53,  1.96s/it]
70it [00:55,  2.06s/it]
71it [00:57,  2.12s/it]
72it [01:00,  2.16s/it]
73it [01:02,  2.19s/it]
74it [01:04,  2.22s/it]
79it [01:06,  1.70s/it]
82it [01:09,  1.41s/it]
88it [01:11,  1.10s/it]
90it [01:13,  1.12s/it]
93it [01:16,  1.01s/it]
94it [01:18,  1.38s/it]
95it [01:20,  1.65s/it]
97it [01:22,  1.50s/it]
98it [01:

0:15:29.383801


In [215]:
count_df

,name,0,1,2,3,4,5,count,owned,target_lus,lu_diff
0,Adam Hadwin,735.0,1.0,1.0,1.0,NaN,7.0,745.0,0.05,750.0,5.0
1,Adam Rainaud,NaN,NaN,NaN,2.0,6.0,NaN,8.0,NaN,NaN,NaN
2,Adam Scott,935.0,41.0,NaN,NaN,NaN,1021.0,1997.0,0.25,3750.0,1753.0
3,Alex Beach,NaN,NaN,NaN,NaN,6.0,NaN,6.0,NaN,NaN,NaN
4,Alexander Levy,130.0,13.0,1.0,5.0,NaN,NaN,149.0,0.01,150.0,1.0
5,Alexander Noren,627.0,76.0,3.0,NaN,NaN,31.0,737.0,0.05,750.0,13.0
6,Andrew Johnston,130.0,21.0,3.0,2.0,1.0,NaN,157.0,0.01,150.0,-7.0
7,Andy Sullivan,125.0,18.0,5.0,1.0,NaN,2.0,151.0,0.01,150.0,-1.0
8,Anirban Lahiri,118.0,23.0,5.0,2.0,1.0,NaN,149.0,0.01,150.0,1.0
9,Bernd Wiesberger,476.0,92.0,7.0,NaN,NaN,20.0,595.0,0.04,600.0,5.0


In [197]:
count_df.loc[count_df['name'] == 'Rory McIlroy']

,name,0,1,2,3,4,5,count,owned,target_lus,lu_diff
122,Rory McIlroy,636.0,24.0,NaN,NaN,NaN,NaN,660.0,0.23,3450.0,-2790.0


In [216]:
base_sql = """SELECT strokes_gained.player_id, strokes_gained.player_name,
    strokes_gained.date_started,
    strokes_gained.tournament_id,
    strokes_gained.course_num,
    strokes_gained.`year`,
    strokes_gained.round_num,
    strokes_gained.hole_num,
    strokes_gained.hole_id, 
    strokes_gained.par,
    strokes_gained.stroke_num, 
    strokes_gained.stroke_gained,
    strokes_gained.shot_loc,
    strokes_gained.shot_dist,
    course_holes.hole_id,
    course_holes.tee_loc_dist_to_pin
    FROM strokes_gained
    inner join course_holes
    on course_holes.hole_id = strokes_gained.hole_id"""
#base_sql = 'SELECT * FROM pga.strokes_gained'
#

for p, pid in enumerate(range(50)):
    pid_formatted = str(np.int64(pid))
    if p < 1:
        where_sql = ' Where player_id = {} or'.format(pid_formatted)
    else:
        where_sql = ' player_id = {} or'.format(pid_formatted)

    base_sql = base_sql + where_sql

In [217]:
base_sql

'SELECT strokes_gained.player_id, strokes_gained.player_name,\n    strokes_gained.date_started,\n    strokes_gained.tournament_id,\n    strokes_gained.course_num,\n    strokes_gained.`year`,\n    strokes_gained.round_num,\n    strokes_gained.hole_num,\n    strokes_gained.hole_id, \n    strokes_gained.par,\n    strokes_gained.stroke_num, \n    strokes_gained.stroke_gained,\n    strokes_gained.shot_loc,\n    strokes_gained.shot_dist,\n    course_holes.hole_id,\n    course_holes.tee_loc_dist_to_pin\n    FROM strokes_gained\n    inner join course_holes\n    on course_holes.hole_id = strokes_gained.hole_id Where player_id = 0 or player_id = 1 or player_id = 2 or player_id = 3 or player_id = 4 or player_id = 5 or player_id = 6 or player_id = 7 or player_id = 8 or player_id = 9 or player_id = 10 or player_id = 11 or player_id = 12 or player_id = 13 or player_id = 14 or player_id = 15 or player_id = 16 or player_id = 17 or player_id = 18 or player_id = 19 or player_id = 20 or player_id = 21 or